# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

sentences = [sentence for sentence in sentences]

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

815775

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

taxes_communale


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

2

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [9]:
%time bigram_phrases[sentences[78]]

CPU times: user 1.16 ms, sys: 33 µs, total: 1.19 ms
Wall time: 1.23 ms


['maladie',
 ',',
 'le',
 'conseil',
 'ajourne',
 'l',
 "'",
 'examen',
 'de',
 'la',
 'reclamation',
 'des',
 'tailleurs',
 'de',
 'pierre',
 'et',
 'marchands',
 'de',
 'briques',
 ',',
 'contre',
 'quelques',
 'articles',
 'des',
 'taxes_communales',
 ',',
 'et',
 'la',
 'discussion',
 'des',
 'budgets',
 'de',
 'la']

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [11]:
%time bigram_phraser[sentences[78]]

CPU times: user 351 µs, sys: 10 µs, total: 361 µs
Wall time: 382 µs


['maladie',
 ',',
 'le',
 'conseil',
 'ajourne',
 'l',
 "'",
 'examen',
 'de',
 'la',
 'reclamation',
 'des',
 'tailleurs',
 'de',
 'pierre',
 'et',
 'marchands',
 'de',
 'briques',
 ',',
 'contre',
 'quelques',
 'articles',
 'des',
 'taxes_communales',
 ',',
 'et',
 'la',
 'discussion',
 'des',
 'budgets',
 'de',
 'la']

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [12]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [13]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [14]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [15]:
print(corpus[:100])

[['v', 'i', 'l', 'l', 'e', 'de'], ['bruxelles'], ['bulletin', 'ires', '8eanas'], ['dl', '!'], ['conseil'], ['communal'], ['annee'], ['1847', '.'], ['au'], ['ville', 'de', 'b', 'r', 'u', 'x', 'e', 'l', 'l', 'e', 's', '.'], ['bulletin'], ['conseil'], ['aes'], ['seances'], ['communal', '.'], ['annee'], ['1847', '.'], ['bruxelles', ','], ['imprimerie'], ['d', 'e', 'j', '.'], ['h', '.', 'b', 'r', 'i', 'a', 'r', 'd', ','], ['rite', 'n', 'e', 'u', 'v', 'e', ',', '3', '1', ',', 'faubourg', 'de', 'n', 'a', 'm', 'u', 'r', ','], ['1', '84', '8'], ['de', '!'], ['du', 'consei'], ['dibi'], ['e', '.', '-', 'communication'], ['conclusions', 'de', 'la', 'section', 'des'], ['du', 'nouvel_hospice', 'pour', 'les', 'av'], ['enraisonde', 'l', "'", 'absence', '&'], ['maladie', '.', 'le', 'conseil', 'ajourne'], ['leurs', 'de', 'pierre', 'el', 'marchai'], ['cles', 'des', 'taxes', 'communale', "'"], ['bieniaance', 'eldeseianv'], ['il', 'est', 'donne', 'communie', ';'], ['mandant', 'le', 'o', 'p', 'fa'], ['gnant

## Entrainement d'un modèle Word2Vec sur ce corpus

In [16]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 1min 37s, sys: 3.73 s, total: 1min 40s
Wall time: 46.9 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [17]:
outfile = f"../data/bulletins.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [18]:
model = Word2Vec.load("../data/bulletins.model")

### Imprimer le vecteur d'un terme

In [19]:
model.wv["bruxelles"]

array([ 0.6685848 , -1.5555533 , -0.24623439, -0.45822918, -0.8627349 ,
       -2.160193  ,  0.7340632 ,  0.2643709 ,  1.4220833 ,  1.363524  ,
       -0.7210393 , -1.7041965 ,  1.0189964 ,  1.2681946 ,  0.6524234 ,
       -0.270122  , -0.13729411, -1.9944715 , -1.8950688 , -1.7195004 ,
       -0.5283786 , -1.9022456 ,  1.1998467 , -2.8916116 , -4.8399153 ,
       -0.6139503 , -3.6063063 , -2.5714958 , -2.4441621 , -1.6549344 ,
       -2.7475863 ,  0.22190203], dtype=float32)

### Calculer la similarité entre deux termes

In [20]:
model.wv.similarity("boucher", "boulanger")

0.719111

### Chercher les mots les plus proches d'un terme donné

In [21]:
model.wv.most_similar("bruxelles", topn=10)

[('nivelles', 0.7111584544181824),
 ('liege', 0.7082246541976929),
 ('hotel', 0.6960926055908203),
 ('paris', 0.6888905167579651),
 ('gand', 0.6448415517807007),
 ('saint_-_gilles', 0.627330482006073),
 ('holel', 0.597263514995575),
 ('louvain', 0.5759913325309753),
 ('paix', 0.5642114281654358),
 ('schaerbeek', 0.5608215928077698)]

### Faire des recherches complexes à travers l'espace vectoriel

In [22]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('nivelles', 0.7726391553878784), ('paris', 0.7106684446334839), ('liege', 0.643231987953186), ('londres', 0.6237877011299133), ('gand', 0.6175819635391235), ('voudrais_pas', 0.6083259582519531), ('hotel', 0.604787290096283), ('holel', 0.5877097845077515), ('saint_-_gilles', 0.5827161073684692), ('vilvorde', 0.5826244950294495)]
